* Spark MLlib Decision tree example

A college dataset (cllege.txt) is used as the input data set.

Problem statement: Try to classify colleges as Private or Public based on the features.

Data dictionary is as below:

School - Name of the college

Private - Has Yes or No indicating if the college is private or public university

Apps - Number of applications received

Accept - Number of applications accepted

Enroll - Number of new students enrolled

Top10perc - Pct. new students from top 10% of H.S. class

Top25perc - Pct. new students from top 25% of H.S. class

F_Undergrad - Number of fulltime undergraduates

P_Undergrad - Number of parttime undergraduates

Outstate - Out-of-state tuition

Room_Board -  Room and board costs

Books - Estimated book costs

Personal - Estimated personal spending

PhD - Percentage of faculty with Ph.D.’s

Terminal - Percentage of faculty with terminal degree

S_F_Ratio - Student vs Faculty ratio

Perc_alumni - Percentage of alumni who donate

Expend - Instructional expenditure per student

Grad_Rate - Graduation rate


In [ ]:
import findspark
findspark.init("C:\spark_setup\spark-3.1.2-bin-hadoop2.7")
import pyspark

In [ ]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Decisiontreeexample').getOrCreate()

In [ ]:
# Load training data which should be in Hadoop at the location /user/root
data = spark.read.csv('college.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

In [ ]:
data.head()

Transforming input dataframe into the format required for Spark MLlib algorithm

In [ ]:
# Dataframe needs to be in the form of two columns
# ("label","features") where features is a vector

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

In [ ]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [ ]:
output = assembler.transform(data)

In [ ]:
type(output)

In [ ]:
output.show(truncate=False)

Data type of Private column is string with "yes" or "no" as the values.

In its place numeric values of 0 or 1 needs to be used. A column PrivateIndex is added for this purpose.

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [ ]:
output_fixed.show(5)

In [ ]:
final_data = output_fixed.select("features",'PrivateIndex')

In [ ]:
final_data.show()

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
final_data.show(10,False)

DecisionTreeClassifier

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

Create the model using default options and specifying impurity as 'entropy'.
Another option for impurity 'gini' can be also used.

In [ ]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features',impurity='entropy')

Train the model

In [ ]:
dtc_model = dtc.fit(train_data)

Test the model with test part of the input data.
Output dataframe contains:

predictionCol "prediction" - Double which is the Predicted label	

rawPredictionCol "rawPrediction" - Vector which is a Vector of length # classes (length 2 in this example), with the counts of training instance labels at the tree node which makes the prediction

probabilityCol "probability" - Vector which is a Vector of length # classes equal to rawPrediction (length 2 in this example), normalized to a multinomial distribution

In [ ]:
dtc_predictions = dtc_model.transform(test_data)

In [ ]:
test_data.show(5,False)

In [ ]:
dtc_predictions.printSchema()

Evaluation Metrics

In [ ]:
dtc_predictions.show(5)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# Select (prediction, true label) and compute test accuracy
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [ ]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)

In [ ]:
print(dtc_acc)

In [ ]:
# Display the value as a percentage with a better formatting
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)

In [ ]:
print(dtc_model.toDebugString)